In [146]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import plotly.express as px
from numba import njit
from plotly.subplots import make_subplots
import math

1. Descargar datos historicos

In [147]:
columns = ['Open' , 'High' , 'Low' , 'Close']
data = vbt.YFData.download(
    ['AUDUSD=X'],
    missing_index = 'drop',
    start = '2010-01-01',
    end = '2026-01-01'
).get(columns)

C:\Users\Evee2\AppData\Local\Temp\ipykernel_7852\1569435725.py:2: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.



2. Definir estrategia (Reversion a la media con bandas de bollinger 'bidireccional')

In [148]:
# indicadores a utilizar
bbands_indicator = vbt.IndicatorFactory.from_talib('BBANDS')
atr_indicator = vbt.IndicatorFactory.from_talib('ATR')

@njit
def generate_signal(close , upper_band , lower_band):

    long_signal = np.where(close > upper_band , -1 , 0)
    long_signal = np.where(close < lower_band , 1 , long_signal)

    short_signal = np.where(close < lower_band , -1 , 0)
    short_signal = np.where(close > upper_band , 1 , short_signal)

    return long_signal , short_signal

def bbands_strategy(open , high , low , close , bands_period , bands_std , atr_period , atr_sl , risk_per_trade):
    
    upper_band = bbands_indicator.run(close , timeperiod=bands_period , nbdevup=bands_std , nbdevdn=bands_std).upperband.to_numpy()
    lower_band = bbands_indicator.run(close , timeperiod=bands_period , nbdevup=bands_std , nbdevdn=bands_std).lowerband.to_numpy()
    atr = atr_indicator.run(high , low , close , timeperiod = atr_period).real.shift(1).to_numpy()
    
    atr_distance = (atr * atr_sl) / open
    
    size = risk_per_trade / atr_distance
    
    long_signal , short_signal = generate_signal(close , upper_band , lower_band)
    
    return long_signal , short_signal , size , atr_distance , upper_band , lower_band


strategy_generator = vbt.IndicatorFactory(
    class_name = 'bbands',
    short_name = 'bb',
    input_names = ['open' , 'high' , 'low' , 'close'],
    param_names = ['bands_period' , 'bands_std' , 'atr_period' , 'atr_sl' , 'risk_per_trade'],
    output_names = ['long_signal' , 'short_signal' , 'size' , 'atr_distance' , 'upper_band' , 'lower_band']
).from_apply_func(
    bbands_strategy,
    bands_period = 20,
    bands_std = 2,
    atr_period = 14,
    atr_sl = 3,
    risk_per_trade = 0.01
)

In [149]:
strategy = strategy_generator.run(data['Open'] , data['High'] , data['Low'] , data['Close'])

long_entries = (strategy.long_signal == 1).vbt.fshift(1)
long_exits = (strategy.long_signal == -1).vbt.fshift(1)

short_entries = (strategy.short_signal == 1).vbt.fshift(1)
short_exits = (strategy.short_signal == -1).vbt.fshift(1)

pf = vbt.Portfolio.from_signals(
    
    init_cash = 50000,
    price = data['Open'],
    close = data['Close'],
    high = data['High'],
    low = data['Low'],
    entries = long_entries,
    exits = long_exits,
    short_entries = short_entries,
    short_exits = short_exits,
    upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Close,
    sl_stop = strategy.atr_distance,
    stop_entry_price = vbt.portfolio.enums.StopEntryPrice.Price,
    stop_exit_price = vbt.portfolio.enums.StopExitPrice.StopMarket,
    size = strategy.size,
    size_type = vbt.portfolio.enums.SizeType.Percent,
    
)

In [150]:
fig = pf.plot()
fig.add_trace(
    strategy.upper_band.vbt.plot(trace_kwargs = dict(name = 'upper_band')).data[0]
)
fig.add_trace(
    strategy.lower_band.vbt.plot(trace_kwargs = dict(name = 'lower_band')).data[0]
)
fig.show()

In [151]:
trades = pf.trades.records_readable
trades

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,0,14944.693733,2010-02-01 00:00:00+00:00,0.885818,0.0,2010-03-10 00:00:00+00:00,0.915583,0.0,444.831629,0.033602,Long,Closed,0
1,1,0,20355.412637,2010-04-06 23:00:00+00:00,0.927816,0.0,2010-05-04 23:00:00+00:00,0.909587,0.0,371.056548,0.019647,Short,Closed,1
2,2,0,18262.959672,2010-05-05 23:00:00+00:00,0.906290,0.0,2010-05-16 23:00:00+00:00,0.877501,0.0,-525.768785,-0.031766,Long,Closed,2
3,3,0,13768.643938,2010-05-18 23:00:00+00:00,0.862366,0.0,2010-05-19 23:00:00+00:00,0.813405,0.0,-674.132918,-0.056776,Long,Closed,3
4,4,0,10871.494078,2010-05-20 23:00:00+00:00,0.813140,0.0,2010-09-15 23:00:00+00:00,0.938791,0.0,1366.008940,0.154525,Long,Closed,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,124,0,31379.276938,2025-07-01 23:00:00+00:00,0.657990,0.0,2025-07-31 23:00:00+00:00,0.642820,0.0,476.017334,0.023055,Short,Closed,124
125,125,0,30243.845591,2025-08-03 23:00:00+00:00,0.647011,0.0,2025-09-09 23:00:00+00:00,0.658588,0.0,350.147729,0.017894,Long,Closed,125
126,126,0,33422.645814,2025-09-11 23:00:00+00:00,0.666560,0.0,2025-10-13 23:00:00+00:00,0.651380,0.0,507.355487,0.022774,Short,Closed,126
127,127,0,35358.859406,2025-10-15 23:00:00+00:00,0.650670,0.0,2025-12-05 00:00:00+00:00,0.660742,0.0,356.127857,0.015479,Long,Closed,127


3. Funcion para determinar ventanas de walk forward

In [152]:
def generar_wfo_unanchored(data , n_runs , os_perc , plot = False):
    
    total_len = len(data)
    
    os_len = math.floor(total_len / (n_runs + (1 / os_perc) -1))
    is_len = math.floor(os_len * ((1 - os_perc) / os_perc))
    
    window_len = is_len + os_len
    
    splits = data.vbt.rolling_split(
        n = n_runs,
        window_len = window_len,
        set_lens = (os_len,),
        left_to_right = False,
        plot = plot
    )
    
    return splits

In [153]:
generar_wfo_unanchored(data , 1 , 0.5 , True).show()

c:\Users\Evee2\anaconda3\envs\pythofinanc\Lib\site-packages\jupyter_client\session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

c:\Users\Evee2\anaconda3\envs\pythofinanc\Lib\site-packages\jupyter_client\session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



In [154]:
n_runs = 10
os_perc = 0.3

(is_open , is_open_dates) , (os_open , os_open_dates) = generar_wfo_unanchored(data['Open'] , n_runs , os_perc, False)
(is_high , is_high_dates) , (os_high , os_high_dates) = generar_wfo_unanchored(data['High'] , n_runs , os_perc, False)
(is_low , is_low_dates) , (os_low , os_low_dates) = generar_wfo_unanchored(data['Low'] , n_runs , os_perc, False)
(is_close , is_close_dates) , (os_close , os_close_dates) = generar_wfo_unanchored(data['Close'] , n_runs , os_perc, False)

4. Funcion para obtener combinacion de parametros

In [155]:
def optimize_params(init_cash , open , high , low , close , bands_period_window , bands_std_window , atr_period_window , atr_sl_window):
    
    strategy = strategy_generator.run(
        open = open,
        high = high,
        low = low,
        close = close,
        bands_period = bands_period_window,
        bands_std = bands_std_window,
        atr_period = atr_period_window,
        atr_sl = atr_sl_window,
        param_product = True
    )
    
    long_entries = (strategy.long_signal == 1).vbt.fshift(1)
    long_exits = (strategy.long_signal == -1).vbt.fshift(1)

    short_entries = (strategy.short_signal == 1).vbt.fshift(1)
    short_exits = (strategy.short_signal == -1).vbt.fshift(1)

    pf = vbt.Portfolio.from_signals(
        
        init_cash = init_cash,
        price = open,
        close = close,
        high = high,
        low = low,
        entries = long_entries,
        exits = long_exits,
        short_entries = short_entries,
        short_exits = short_exits,
        upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Close,
        sl_stop = strategy.atr_distance,
        stop_entry_price = vbt.portfolio.enums.StopEntryPrice.Price,
        stop_exit_price = vbt.portfolio.enums.StopExitPrice.StopMarket,
        size = strategy.size,
        size_type = vbt.portfolio.enums.SizeType.Percent,
        
    )
    
    return pf.total_return() / -pf.max_drawdown()

In [156]:
bands_period_window = np.arange(10 , 50 , 5 , dtype = int)
bands_std_window = np.arange(2 , 3.5 , 0.5 , dtype = float)
atr_period_window = 14
atr_sl_window = np.arange(2 , 5 , 0.5 , dtype = float)


combs = optimize_params(50000 , is_open , is_high , is_low , is_close , bands_period_window , bands_std_window , atr_period_window , atr_sl_window)

In [157]:
best_index = combs[combs.groupby('split_idx').idxmax()].index

def get_best_params(best_index , level_name):
    return best_index.get_level_values(level_name).to_numpy()

in_best_bands_period = get_best_params(best_index , 'bb_bands_period')
in_best_bands_std = get_best_params(best_index , 'bb_bands_std')
in_best_atr_period = get_best_params(best_index , 'bb_atr_period')
in_best_atr_sl = get_best_params(best_index , 'bb_atr_sl')

in_best_windows_pair = np.array(list(zip(in_best_bands_period , in_best_bands_std , in_best_atr_period , in_best_atr_sl)))

best_params_df = pd.DataFrame(in_best_windows_pair , columns = ['bb_bands_period' , 'bb_bands_std' , 'bb_atr_period' , 'bb_atr_sl'])

fig = make_subplots(
    rows = 2,
    cols = 2
)
fig.add_trace(
    best_params_df['bb_bands_period'].vbt.plot(trace_kwargs = dict(name = 'bands_period')).data[0],
    row = 1,
    col = 1
)
fig.add_trace(
    best_params_df['bb_bands_std'].vbt.plot(trace_kwargs = dict(name = 'bands_std')).data[0],
    row = 1,
    col = 2
)
fig.add_trace(
    best_params_df['bb_atr_period'].vbt.plot(trace_kwargs = dict(name = 'atr_period')).data[0],
    row = 2,
    col = 1
)
fig.add_trace(
    best_params_df['bb_atr_sl'].vbt.plot(trace_kwargs = dict(name = 'atr_sl')).data[0],
    row = 2,
    col = 2
)
fig.show()




In [158]:
def simulate_best_params(init_cash , open , high , low , close , best_bands_period , best_bands_std , best_atr_period , best_atr_sl):
    
    strategy = strategy_generator.run(
        open = open,
        high = high,
        low = low,
        close = close,
        bands_period = best_bands_period,
        bands_std = best_bands_std,
        atr_period = best_atr_period,
        atr_sl = best_atr_sl,
        per_column = True
    )
    
    long_entries = (strategy.long_signal == 1).vbt.fshift(1)
    long_exits = (strategy.long_signal == -1).vbt.fshift(1)

    short_entries = (strategy.short_signal == 1).vbt.fshift(1)
    short_exits = (strategy.short_signal == -1).vbt.fshift(1)

    pf = vbt.Portfolio.from_signals(
        
        init_cash = init_cash,
        price = open,
        close = close,
        high = high,
        low = low,
        entries = long_entries,
        exits = long_exits,
        short_entries = short_entries,
        short_exits = short_exits,
        upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Close,
        sl_stop = strategy.atr_distance,
        stop_entry_price = vbt.portfolio.enums.StopEntryPrice.Price,
        stop_exit_price = vbt.portfolio.enums.StopExitPrice.StopMarket,
        size = strategy.size,
        size_type = vbt.portfolio.enums.SizeType.Percent,
        
    )
    
    return pf

In [163]:
is_results = simulate_best_params(50000 , is_open , is_high , is_low , is_close , in_best_bands_period , in_best_bands_std , in_best_atr_period , in_best_atr_sl)
is_results.total_return() / -is_results.max_drawdown()

bb_bands_period  bb_bands_std  bb_atr_period  bb_atr_sl  split_idx
20               2.5           14             4.0        0            5.295112
35               2.0           14             2.5        1            3.628796
10               2.5           14             2.0        2            3.407663
                                                         3            3.407663
45               2.5           14             4.0        4            3.100110
10               2.0           14             2.0        5            4.080615
25               2.5           14             4.0        6            4.701509
40               2.5           14             2.5        7            5.140274
45               2.0           14             2.0        8            4.928648
40               2.5           14             3.0        9            7.930001
dtype: float64

In [164]:
os_results = simulate_best_params(50000 , os_open , os_high , os_low , os_close , in_best_bands_period , in_best_bands_std , in_best_atr_period , in_best_atr_sl)
os_results.total_return() / -is_results.max_drawdown()

bb_bands_period  bb_bands_std  bb_atr_period  bb_atr_sl  split_idx
20               2.5           14             4.0        0           -1.584135
35               2.0           14             2.5        1           -1.650648
10               2.5           14             2.0        2           -0.264217
                                                         3           -0.107114
45               2.5           14             4.0        4           -0.601560
10               2.0           14             2.0        5           -1.917552
25               2.5           14             4.0        6           -0.618281
40               2.5           14             2.5        7           -1.144386
45               2.0           14             2.0        8            1.104170
40               2.5           14             3.0        9            4.722859
dtype: float64

In [173]:
returns_concatenated = []
for i in range(len(is_results.total_return())):
    concatenated = pd.concat([is_results.iloc[i].total_return(), os_results.iloc[i].total_return()], axis=0)
    returns_concatenated.append(concatenated)

TypeError: cannot concatenate object of type '<class 'numpy.float64'>'; only Series and DataFrame objs are valid